<a href="https://colab.research.google.com/github/Maagnitude/CRC_slides_models/blob/main/CRC_slides_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from keras import layers
import keras.datasets

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp '/content/drive/MyDrive/Colab Notebooks/CRC_slides.tar.gz' .
!tar -xvzf 'CRC_slides.tar.gz'
data_dir = '/content/CRC_slides'

In [ ]:
data_dir.

In [ ]:
def load_dataset(data_dir, train_pct=0.6, val_pct=0.2, test_pct=0.2, batch_size=64, img_size=(224, 224)):
    
  train_ds = keras.utils.image_dataset_from_directory(data_dir, labels='inferred', label_mode='int', class_name='rgb', 
      batch_size=batch_size, image_size=img_size, shuffle=True, seed=123, validation_split=val_pct, subset='training')
  
  val_ds = keras.utils.image_dataset_from_directory(data_dir, labels='inferred', label_mode='int', class_name='rgb', 
    batch_size=batch_size, image_size=img_size, shuffle=True, seed=123, validation_split=val_pct, subset='validation')
  
  test_ds = keras.utils.image_dataset_from_directory(data_dir, labels='inferred', label_mode='int', class_name='rgb', 
    batch_size=batch_size, image_size=img_size, shuffle=True, seed=123, validation_split=val_pct, subset='test')

  devel_ds = train_ds + val_ds

  classes = train_ds.class_names

  return devel_ds, train_ds, val_ds, test_ds, classes

In [ ]:
def cnn(num_classes):
  